In [16]:
import pandas as pd
import funzioni as fx
import numpy as np
import yfinance as yf
from tensorflow.keras.models import load_model
from tensorflow_addons.metrics import F1Score
from multiprocessing import Pool, cpu_count, Manager, Value
import tensorflow as tf
from datetime import datetime, timedelta, date
import json
import os
import ctypes
from functools import partial
from typing import Tuple

class Trade:
    def __init__(self, nome_ticker, data_inizio, data_fine, importo_da_investire, probabilità_per_acquisto, stop_loss, take_profit, giorni_max_posizione) -> None:
        self.SIMBOLO = nome_ticker
        self.DATA_INIZIO = data_inizio
        self.DATA_FINE = data_fine
        self.BILANCIO_INIZIALE = importo_da_investire
        self.PROBABILITA_PER_ACQUISTO = probabilità_per_acquisto
        self.SL = stop_loss
        self.TP = take_profit
        self.GIORNI_POS = giorni_max_posizione
        self.log = None
        self.model = None
        self.valutazione_modello = None
        self.posizioni = None
        self.bilancio = None
        self.esito = None
        
    def avvia_trading(self, verbose=0) -> None:
        self.log = "\033[42m" + self.SIMBOLO + "\033[0m" + "\n"
        ticker = yf.download(self.SIMBOLO, start=self.DATA_INIZIO, end=self.DATA_FINE, progress=False)
        ticker.index = ticker.index.date
        str = f"Calcolo indicatori ticker {self.SIMBOLO}"
        if verbose == 1: print(str)
        self.log += str + "\n"
        ticker = fx.crea_indicatori(ticker)
        ticker = fx.imposta_target(ticker)
        ticker.dropna(axis=0, inplace=True)

        str = f"Definizione features e target {self.SIMBOLO}"
        if verbose == 1: print(str)
        self.log += str + "\n"        
        idx, X, Y, _ = fx.to_XY(ticker, fx.features_prezzo, fx.features_da_scalare_singolarmente, fx.features_meno_piu, fx.features_candele, fx.features_no_scala, fx.elenco_targets, fx.n_timesteps, fx.giorni_previsione, bilanciamento=0)
        str = f"Previsione {self.SIMBOLO}"
        if verbose == 1: print(str)
        self.log += str + "\n"        
        f1_score = F1Score(num_classes=1, average='macro', threshold=self.PROBABILITA_PER_ACQUISTO)
        self.model = load_model("model.h5", custom_objects={'Addons>F1Score': f1_score})
        y_pred = self.model.predict(X)
        self.valutazione_modello = self.model.evaluate(X, Y)

        X = X[:, -1, :]

        df_X = ticker.loc[ticker.index.intersection(idx)]
        y_pred = y_pred.reshape(-1,)
        df_y_pred = pd.DataFrame(y_pred, columns=['prob_True'], index=idx)
        self.dati = pd.concat([df_X, df_y_pred], axis=1)

        self.bilancio = self.BILANCIO_INIZIALE
        pos_aperta = False
        tp = sl = n_azioni = prezzo_acquisto = prezzo_tot = 0
        giorni_posizione = 0
        self.posizioni = pd.DataFrame(columns=["Bilancio", "Direzione", "Prezzo_un", "n_azioni", "Prezzo_tot", "TP", "SL", "Open", "High", "Low", "Esito", "P_L"], index=self.dati.index)

        for idx, row in self.dati.iterrows():
            if self.bilancio <= 0:
                break
            if pos_aperta == False:
                if row["prob_True"] > self.PROBABILITA_PER_ACQUISTO and self.bilancio > 0:
                    #COMPRA
                    prezzo_acquisto = row["Open"]
                    n_azioni = self.bilancio // prezzo_acquisto
                    prezzo_tot = n_azioni * prezzo_acquisto  
                    self.bilancio -= prezzo_tot          
                    tp = prezzo_acquisto * (1 + self.TP)
                    sl = prezzo_acquisto * (1 - self.SL)
                    pos_aperta = True
                    giorni_posizione = 0
                    self.posizioni.loc[idx] = {"Bilancio": self.bilancio, "Direzione": "COMPRA", "Prezzo_un": prezzo_acquisto, "n_azioni": n_azioni, "Prezzo_tot": -prezzo_tot, "TP": tp, "SL": sl, "Open": row["Open"], "High": row["High"], "Low": row["Low"],}
            else:
                if giorni_posizione == self.GIORNI_POS:
                    prezzo_tot_vendita = row['Close'] * n_azioni
                    self.bilancio += prezzo_tot_vendita
                    pos_aperta = False
                    if prezzo_tot_vendita > prezzo_tot:
                        esito = 'VINCITA'
                    else:
                        esito = 'PERDITA'
                    pl = round(fx.pct_change(prezzo_tot, prezzo_tot_vendita), 0)
                    self.posizioni.loc[idx] = {"Bilancio": self.bilancio, "Direzione": "VENDI", "Prezzo_un": row['Close'], "n_azioni": n_azioni, "Prezzo_tot": prezzo_tot, "Esito": esito, "Open": row["Open"], "High": row["High"], "Low": row["Low"], "P_L": pl}
                if row["High"] >= tp:
                    prezzo_tot_vendita = tp * n_azioni
                    self.bilancio += prezzo_tot_vendita
                    pos_aperta = False
                    giorni_posizione = 0
                    pl = round(fx.pct_change(prezzo_tot, prezzo_tot_vendita), 0)
                    self.posizioni.loc[idx] = {"Bilancio": self.bilancio, "Direzione": "VENDI", "Prezzo_un": tp, "n_azioni": n_azioni, "Prezzo_tot": prezzo_tot, "Esito": "VINCITA", "Open": row["Open"], "High": row["High"], "Low": row["Low"], "P_L": pl}
                elif row["Low"] <= sl:
                    prezzo_tot_vendita = sl * n_azioni
                    self.bilancio += prezzo_tot_vendita
                    pos_aperta = False
                    giorni_posizione = 0
                    pl = round(fx.pct_change(prezzo_tot, prezzo_tot_vendita), 0)
                    self.posizioni.loc[idx] = {"Bilancio": self.bilancio, "Direzione": "VENDI", "Prezzo_un": sl, "n_azioni": n_azioni, "Prezzo_tot": prezzo_tot, "Esito": "PERDITA", "Open": row["Open"], "High": row["High"], "Low": row["Low"], "P_L": pl}
                else:
                    giorni_posizione += 1

        if pos_aperta: 
            self.bilancio += prezzo_tot

        tot_vincite = self.posizioni.loc[self.posizioni["Esito"] == "VINCITA", "Esito"].count()
        tot_perdite = self.posizioni.loc[self.posizioni["Esito"] == "PERDITA", "Esito"].count()
        max_vincita_value = self.posizioni.loc[self.posizioni["Esito"] == "VINCITA", "P_L"].max(skipna=True)
        max_perdita_value = self.posizioni.loc[self.posizioni["Esito"] == "PERDITA", "P_L"].min(skipna=True)
        max_vincita = int(max_vincita_value if max_vincita_value == max_vincita_value else 0)  # max_vincita_value == max_vincita_value è un modo per controllare se non è NaN
        max_perdita = int(max_perdita_value if max_perdita_value == max_perdita_value else 0)
        tot = int(fx.pct_change(self.BILANCIO_INIZIALE, self.bilancio))
        self.esito = (
            f"\nBilancio {self.SIMBOLO} = {round(self.bilancio, 2)}" + "\n" +
            f'Vincite: {tot_vincite}, vincita max: {max_vincita} %' + "\n" + 
            f'Perdite: {tot_perdite}, perdita max: {max_perdita} %' + "\n" +
            f'Totale: {tot} %'
        )
        if verbose == 1: print(self.esito)
        self.log += self.esito + "\n"

class Borsa:
    def __init__(self, n_simboli_contemporanei, bilancio_iniziale, probabilità_per_acquisto, stop_loss, take_profit, giorni_max_posizione, data_inizio=pd.Timestamp(year=1970, month=1, day=1), data_fine=pd.Timestamp.today()):
        self.N_SIMBOLI = n_simboli_contemporanei
        self.DATA_INIZIO = pd.to_datetime(data_inizio)
        self.DATA_FINE = pd.to_datetime(data_fine)
        self.BILANCIO_INIZIALE = bilancio_iniziale
        self.PROBABILITA_PER_ACQUISTO = probabilità_per_acquisto
        self.SL = stop_loss
        self.TP = take_profit
        self.GIORNI_POS = giorni_max_posizione
        self.log = None
        self.f1_score = F1Score(num_classes=1, average='macro', threshold=self.PROBABILITA_PER_ACQUISTO)
        self.model = load_model("model.h5", custom_objects={'Addons>F1Score': self.f1_score})
        self.posizioni = []
        self.bilancio = self.BILANCIO_INIZIALE
        self.lista_tickers = pd.read_parquet("lista_ticker.parquet")
        self.screener = pd.DataFrame(columns=["Ticker", "Prediction"])
    
        if os.path.exists(f'tickers/_indice.json'):
            with open(f'tickers/_indice.json', 'r') as jsonfile:
                indice = json.load(jsonfile)
            prima_data_str = indice['prima_data']
            ultima_data_str = indice['ultima_data']
            prima_data = pd.to_datetime(prima_data_str)
            ultima_data = pd.to_datetime(ultima_data_str)
            if (self.DATA_INIZIO < prima_data) or (self.DATA_FINE > ultima_data):
                min_data = min(prima_data, self.DATA_INIZIO)
                max_data = max(ultima_data, self.DATA_FINE)
                self.scarica_tickers(min_data, max_data, append=True)
        else:
            indice = {
                'prima_data': '1970-01-01',
                'ultima_data': '1970-01-01'
            }
            with open(f'tickers/_indice.json', 'w') as jsonfile:
                json.dump(indice, jsonfile, indent=4)        
            self.scarica_tickers(self.DATA_INIZIO, self.DATA_FINE, append=False)

    def scarica_tickers(self, data_inizio, data_fine, append=False) -> None: 
        tot_tickers = len(self.lista_tickers)
        totale_processati = Value('i', 0)
        data_min = pd.Timestamp.today() #imposta la data minima dei dati scaricati alla data massima possibile
        ts_data_min = Value(ctypes.c_double, data_min.timestamp())
        data_max = pd.Timestamp(1970, 1, 1) #imposta alla data minima possibile
        ts_data_max = Value(ctypes.c_double, data_max.timestamp())
        with Pool(cpu_count()) as p:
            callback_with_args = partial(_callback_tickers, totale_processati=totale_processati, tot_tickers=tot_tickers, ts_data_min=ts_data_min, ts_data_max=ts_data_max)
            for i in range(0, tot_tickers):
                nome_simbolo = self.lista_tickers.iloc[i]["Ticker"]
                param = (nome_simbolo, data_inizio, data_fine, append)
                p.apply_async(_scarica, args=param, callback=callback_with_args)
            p.close()
            p.join()
        data_min = pd.Timestamp.fromtimestamp(ts_data_min.value).strftime('%Y-%m-%d')
        data_max = pd.Timestamp.fromtimestamp(ts_data_max.value).strftime('%Y-%m-%d')
        indice = {
            'prima_data': data_min,
            'ultima_data': data_max
        }
        with open(f'tickers/_indice.json', 'w') as jsonfile:
            json.dump(indice, jsonfile, indent=4)        

    def genera_xy(self, n_simboli=0, casuale=0, bilanciamento=0) -> None:
        tot_tickers = len(self.lista_tickers) if n_simboli == 0 else n_simboli
        if casuale == 1:
            self.lista_tickers = self.lista_tickers.sample(frac=1)
        totale_processati = Value('i', 0)
        with Pool(cpu_count()) as p:
            callback_with_args = partial(_callback_xy, totale_processati=totale_processati, tot_tickers=tot_tickers)
            for i in range(totale_processati.value, tot_tickers):
                nome_simbolo = self.lista_tickers.iloc[i]["Ticker"]
                param = (nome_simbolo, bilanciamento)
                p.apply_async(_genera, args=param, callback=callback_with_args)

            p.close()
            p.join()

    def avvia_screener(self, data='', verbose=1) -> None:
        if data == '':
            data = pd.to_datetime(pd.Timestamp.today())
        else:
            data = pd.to_datetime(data)
        data_iniziale = data - pd.Timedelta(days=365)

        lista = []
        tot_elem = len(self.lista_tickers)

        with Pool(cpu_count()) as pool:
            ticker_infos = [(ticker, data_iniziale, data) for ticker in self.lista_tickers["Ticker"]]
            results = pool.map(_screener_carica_dati, ticker_infos)
            ticker_data = [result for result in results if result is not None]

            if verbose == 1:
                print(f'Numero di ticker da elaborare: {len(ticker_data)}')

            results = pool.map(_screener_elabora_ticker, ticker_data)

        lista = results

        for i, (nome_simbolo, pred) in enumerate(lista):
            print(f'{i+1}/{len(lista)}) Ticker {nome_simbolo}: Predizione {pred}')

def _screener_carica_dati(self, nome_simbolo, data_iniziale, data):
    ticker = pd.read_parquet(f'tickers/{nome_simbolo}.parquet')

    if not ticker.empty:
        ticker = ticker[(ticker.index >= data_iniziale) & (ticker.index <= data)]
        _, X, Y, _ = fx.to_XY(ticker, bilanciamento=0)
        return nome_simbolo, X

def _screener_elabora_ticker(self, ticker_info):
    try:
        nome_simbolo, X = ticker_info
        pred = self.model.predict(X)  
        return nome_simbolo, pred[-1, 0]
    except Exception as e:
        return nome_simbolo, str(e)


def _genera(nome_simbolo, bilanciamento):
    try:
        ticker = pd.read_parquet(f"tickers/{nome_simbolo}.parquet")
        _, X, Y, _ = fx.to_XY(ticker, bilanciamento)
        return nome_simbolo, X, Y, ""
    except Exception as e:
        return nome_simbolo, None, None, str(e)
def _callback_xy(result, totale_processati, tot_tickers, perc_base):
    nome_simbolo, X, Y, err = result
    if err == '':
        print(f'X.shape:{X.shape}')
        print(f'Y.shape:{Y.shape}')
        print("Salvataggio su file")
        np.save(f'xy/{nome_simbolo}_X', X)
        np.save(f'xy/{nome_simbolo}_Y', Y)
        print(f"{totale_processati.value}/{tot_tickers}) Completato ticker {nome_simbolo}")
    else:
        print(err)
    with totale_processati.get_lock(): 
        totale_processati.value += 1
def _scarica(nome_simbolo, data_inizio, data_fine, append):
    try:
        if append == True:
            ticker = pd.read_parquet(f'tickers/{nome_simbolo}.parquet')
            ti_min = ticker.index.min()
            ti_max = ticker.index.max()
            if data_inizio < ti_min:
                inizio = data_inizio - pd.Timedelta(days=365)
                fine = ti_min - pd.Timedelta(days=1)
                df_inizio = fx.analizza_ticker(nome_simbolo, start=inizio, end=fine, progress=False, dropna_iniziali=True, dropna_finali=False)
                df_inizio = df_inizio[df_inizio.index >= data_inizio]
                ticker = pd.concat([df_inizio, ticker], axis=0, ignore_index=False)
            if data_fine > ti_max:
                inizio = ti_max - pd.Timedelta(days=365) 
                fine = data_fine
                df_fine = fx.analizza_ticker(nome_simbolo, start=inizio, end=fine, progress=False, dropna_iniziali=True, dropna_finali=False)
                ticker = ticker[ticker.index < df_fine.index.min()]
                ticker = pd.concat([ticker, df_fine], axis=0, ignore_index=False)
        else:
            ticker = fx.analizza_ticker(nome_simbolo, start=data_inizio, end=data_fine, progress=False, dropna_iniziali=True, dropna_finali=False)
        return nome_simbolo, ticker, ""
    except Exception as e:
        return nome_simbolo, None, str(e)
def _callback_tickers(result, totale_processati, tot_tickers, ts_data_min, ts_data_max):
    nome_simbolo, ticker, error = result
    if error == "":
        ticker.to_parquet(f'tickers/{nome_simbolo}.parquet')
        with ts_data_min.get_lock():
            try:
                d = ticker.index.min().timestamp()
                #dt = datetime(d.year, d.month, d.day, hour=0, minute=0, second=0).timestamp()
                if d < ts_data_min.value:
                    ts_data_min.value = d
            except Exception as e:
                print(f"Errore su funzione di callback di scarico ticker {nome_simbolo}: {str(e)}")
                
        with ts_data_max.get_lock():
            try:
                d = ticker.index.max().timestamp()
                #dt = datetime(d.year, d.month, d.day, hour=0, minute=0, second=0).timestamp()
                if d > ts_data_max.value:
                    ts_data_max.value = d
            except Exception as e:
                print(f"Errore su {nome_simbolo}: {str(e)}")

    else:
        print(f"Errore per {nome_simbolo}: {error}")

    with totale_processati.get_lock(): 
        totale_processati.value += 1
    print(f"{totale_processati.value}/{tot_tickers}) Completato ticker {nome_simbolo}")   


In [17]:
if __name__ == "__main__":
    TP = 1
    SL = 0.01
    PROBABILITA_PER_ACQUISTO = 0.5
    BILANCIO_INIZIALE = 1000
    DATA_INIZIO = '2005-01-01'
    DATA_FINE = '2023-12-31'
    GIORNI_MAX_POSIZIONE = 20
    N_SIMBOLI = 10
    borsa = Borsa(N_SIMBOLI, BILANCIO_INIZIALE, PROBABILITA_PER_ACQUISTO, SL, TP, GIORNI_MAX_POSIZIONE)
    borsa.avvia_screener()
    

Errore per ZIP: 'NoneType' object has no attribute 'name'
1/2581) Completato ticker ZIP
Errore per CE: 'NoneType' object has no attribute 'name'
2/2581) Completato ticker CE
Errore per PPBI: 'NoneType' object has no attribute 'name'
3/2581) Completato ticker PPBI
Errore per CHEF: 'NoneType' object has no attribute 'name'
4/2581) Completato ticker CHEF
Errore per YPF: 'NoneType' object has no attribute 'name'
5/2581) Completato ticker YPF
Errore per IRTC: 'NoneType' object has no attribute 'name'
6/2581) Completato ticker IRTC
Errore per NGVT: 'NoneType' object has no attribute 'name'
7/2581) Completato ticker NGVT
Errore per DV: 'NoneType' object has no attribute 'name'
8/2581) Completato ticker DV
Errore per CARR: 'NoneType' object has no attribute 'name'
9/2581) Completato ticker CARR
Errore per SCPL: 'NoneType' object has no attribute 'name'
10/2581) Completato ticker SCPL
Errore per PII: 'NoneType' object has no attribute 'name'
11/2581) Completato ticker PII
Errore per NABL: 'None


1 Failed download:
['INW']: Exception('%ticker%: No price data found, symbol may be delisted (1d 1969-01-01 00:00:00 -> 2011-07-19 00:00:00)')


Errore per REXR: 'NoneType' object has no attribute 'name'
56/2581) Completato ticker REXR
Errore per BERY: 'NoneType' object has no attribute 'name'
57/2581) Completato ticker BERY
Errore per INW: single positional indexer is out-of-bounds
58/2581) Completato ticker INW
Errore per BAK: 'NoneType' object has no attribute 'name'
59/2581) Completato ticker BAK
Errore per WABC: index 0 is out of bounds for axis 0 with size 0
60/2581) Completato ticker WABC
Errore per OHI: 'NoneType' object has no attribute 'name'
61/2581) Completato ticker OHI
Errore per LOW: index 0 is out of bounds for axis 0 with size 0
62/2581) Completato ticker LOW
Errore per IIIN: 'NoneType' object has no attribute 'name'
63/2581) Completato ticker IIIN
Errore per BANC: 'NoneType' object has no attribute 'name'
64/2581) Completato ticker BANC
Errore per KFY: 'NoneType' object has no attribute 'name'
65/2581) Completato ticker KFY
Errore per INFA: 'NoneType' object has no attribute 'name'
66/2581) Completato ticker I


1 Failed download:
['REY']: Exception('%ticker%: No price data found, symbol may be delisted (1d 1969-01-01 00:00:00 -> 2005-05-24 00:00:00)')


Errore per WTW: 'NoneType' object has no attribute 'name'
95/2581) Completato ticker WTW
Errore per ELN: [Errno 2] No such file or directory: 'tickers/ELN.parquet'
96/2581) Completato ticker ELN
Errore per REY: single positional indexer is out-of-bounds
97/2581) Completato ticker REY
Errore per ELF: 'NoneType' object has no attribute 'name'
98/2581) Completato ticker ELF
Errore per MEI: index 0 is out of bounds for axis 0 with size 0
99/2581) Completato ticker MEI
Errore per PLYM: 'NoneType' object has no attribute 'name'
100/2581) Completato ticker PLYM
Errore per WPP: index 0 is out of bounds for axis 0 with size 0
101/2581) Completato ticker WPP
Errore per RMD: 'NoneType' object has no attribute 'name'
102/2581) Completato ticker RMD
Errore per TW: 'NoneType' object has no attribute 'name'
103/2581) Completato ticker TW
Errore per EC: 'NoneType' object has no attribute 'name'
104/2581) Completato ticker EC
Errore per BRFS: 'NoneType' object has no attribute 'name'
105/2581) Completa

...

Errore per PIA: single positional indexer is out-of-bounds
398/2581) Completato ticker PIA
Errore per GTY: index 0 is out of bounds for axis 0 with size 0
399/2581) Completato ticker GTY
Errore per HAYW: 'NoneType' object has no attribute 'name'
400/2581) Completato ticker HAYW
Errore per SLM: index 0 is out of bounds for axis 0 with size 0
401/2581) Completato ticker SLM
Errore per HCCI: 'NoneType' object has no attribute 'name'
402/2581) Completato ticker HCCI
Errore per DOW: 'NoneType' object has no attribute 'name'
403/2581) Completato ticker DOW
Errore per IMAX: 'NoneType' object has no attribute 'name'
404/2581) Completato ticker IMAX
Errore per GIB: 'NoneType' object has no attribute 'name'
405/2581) Completato ticker GIB
Errore per PCOM: [Errno 2] No such file or directory: 'tickers/PCOM.parquet'
406/2581) Completato ticker PCOM
Errore per ASAI: 'NoneType' object has no attribute 'name'
407/2581) Completato ticker ASAI
Errore per MCY: index 0 is out of bounds for axis 0 with si


1 Failed download:
['UCG']: Exception('%ticker%: No price data found, symbol may be delisted (1d 1969-01-01 00:00:00 -> 2005-05-24 00:00:00)')


Errore per AU: 'NoneType' object has no attribute 'name'
476/2581) Completato ticker AU
Errore per FUTU: 'NoneType' object has no attribute 'name'
477/2581) Completato ticker FUTU
Errore per CPG: index 0 is out of bounds for axis 0 with size 0
478/2581) Completato ticker CPG
Errore per GOLD: index 0 is out of bounds for axis 0 with size 0
479/2581) Completato ticker GOLD
Errore per UCG: single positional indexer is out-of-bounds
480/2581) Completato ticker UCG
Errore per BG: 'NoneType' object has no attribute 'name'
481/2581) Completato ticker BG
Errore per IP: index 0 is out of bounds for axis 0 with size 0
482/2581) Completato ticker IP
Errore per MIR: 'NoneType' object has no attribute 'name'
483/2581) Completato ticker MIR
Errore per ABT: index 0 is out of bounds for axis 0 with size 0
484/2581) Completato ticker ABT
Errore per CPB: index 0 is out of bounds for axis 0 with size 0
485/2581) Completato ticker CPB
Errore per COLL: 'NoneType' object has no attribute 'name'
486/2581) Co


1 Failed download:
['SCF']: Exception('%ticker%: No price data found, symbol may be delisted (1d 1969-01-01 00:00:00 -> 2011-01-09 00:00:00)')


Errore per OTEX: 'NoneType' object has no attribute 'name'
775/2581) Completato ticker OTEX
Errore per BLDR: 'NoneType' object has no attribute 'name'
776/2581) Completato ticker BLDR
Errore per MLP: index 0 is out of bounds for axis 0 with size 0
777/2581) Completato ticker MLP
Errore per WTBA: 'NoneType' object has no attribute 'name'
778/2581) Completato ticker WTBA
Errore per SIX: 'NoneType' object has no attribute 'name'
779/2581) Completato ticker SIX
Errore per SCF: single positional indexer is out-of-bounds
780/2581) Completato ticker SCF
Errore per FDP: 'NoneType' object has no attribute 'name'
781/2581) Completato ticker FDP
Errore per MOG^A: [Errno 2] No such file or directory: 'tickers/MOG^A.parquet'
782/2581) Completato ticker MOG^A
Errore per NETI: 'NoneType' object has no attribute 'name'
783/2581) Completato ticker NETI
Errore per PDD: 'NoneType' object has no attribute 'name'
784/2581) Completato ticker PDD
Errore per CCB: 'NoneType' object has no attribute 'name'
785/

In [18]:
borsa.screener

,Ticker,Prediction
